In [287]:
from scripts.models import VehTraj, PedTraj
from esper.widget import vgrid_widget
from vgrid import VideoBlockFormat, NestedFormat, NamedIntervalSet, IntervalBlock
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.poutine import trace
import math
from tqdm import tqdm_notebook as tqdm
from scripts.utils import img_grid, vgrid_traj, make_batch
import pandas as pd
from rekall import IntervalSetMapping, IntervalSet, Interval, Bounds3D
from enum import Enum
from torch import tensor
import torch
from pyro import plate
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO, JitTraceEnum_ELBO, config_enumerate
from pyro.optim import Adam
from pprint import pprint
from torch.distributions import constraints
from copy import copy
import re
pyro.enable_validation(True)

In [288]:
vehicles = list(Vehicle.objects.all())
veh_trajectories = [VehTraj(veh) for veh in vehicles]

pedestrians = list(Pedestrian.objects.all())
ped_trajectories = [PedTraj(ped) for ped in pedestrians]

In [519]:
class MoveState(Enum):
    Moving = 0
    Slowing = 1
    Stopped = 2
    Speeding = 3
        
    def classify(vel_seq):
        if vel_seq.mean() <= 0.04:
            return MoveState.Stopped
        elif vel_seq.std() <= 0.015:
            return MoveState.Moving
        elif vel_seq[0] >= vel_seq[-1]:
            return MoveState.Slowing
        else:
            return MoveState.Speeding  
   
    def to_char(self):
        if self == MoveState.Moving:
            return 'o'
        elif self == MoveState.Slowing:
            return '<'
        elif self == MoveState.Stopped:
            return '_'
        else:
            return '>'
        
    @classmethod
    def model(cls, N):
        ncat = len(cls)
        state = cls(pyro.sample(
            'state', dist.Categorical(torch.full((ncat,), 1/ncat)), infer={"enumerate": "sequential"}).int().item())
        mean = pyro.sample('p_vel_mean', dist.Uniform(0.04, 1.))
        start = pyro.sample('p_slow_start', dist.Uniform(0., 1.))
        length = 1 - pyro.sample('p_slow_length', dist.Exponential(5.))
        start = pyro.sample('p_speed_start', dist.Uniform(0., 1.))
        length = 1 - pyro.sample('p_speed_length', dist.Exponential(5.))
        if state == MoveState.Stopped:
            return [pyro.sample('vel_seq_{}'.format(i), dist.Normal(0, 0.02))
                    for i in pyro.plate('stopped_plate', N)]
        elif state == MoveState.Moving:
            return [pyro.sample('vel_seq_{}'.format(i), dist.Normal(mean, 0.02))
                    for i in pyro.plate('moving_plate', N)]
        elif state == MoveState.Slowing:

            #length = 1 - pyro.sample('p_slow_length', dist.Exponential(5.))
            return [pyro.sample('vel_seq_{}'.format(i), dist.Normal(start - length * i / N, 0.01)) 
                    for i in pyro.markov(range(N))]
        elif state == MoveState.Speeding:

            return [pyro.sample('vel_seq_{}'.format(i), dist.Normal(start + length * i / N, 0.01)) 
                    for i in pyro.markov(range(N))]

In [449]:
def compute_window_size(video):
    return int(video.fps/2)
    
batches = []
batches_flat = []
batch_traj = []
batch_traj_flat = []
for traj in veh_trajectories:
    points = traj.for_numpy()
    window_size = compute_window_size(traj.obj.video)
    vels = points[::3][1:] - points[::3][:-1]
    vel_mag = np.linalg.norm(vels, axis=1)
    batches.append(signal_batch(vel_mag, window_size))
    batch_traj.append([traj.index(lambda t: t[i*window_size*3:(i+1)*window_size*3]) for i in range(len(batches[-1]))])

batches_flat = [b for l in batches for b in l]
batch_traj_flat = [t for l in batch_traj for t in l]

pred_classes = [
    [MoveState.classify(batch) for batch in l]
    for l in tqdm(batches)
]
pred_classes_flat = [p for l in pred_classes for p in l]

In [537]:
from pyro.contrib.autoguide import AutoGuideList, AutoDelta
from scripts.utils import AutoDiscreteSequential
from pyro import poutine
from torch.distributions import constraints

state_posterior = []
for batch in tqdm(batches_flat[:1]):
    pyro.clear_param_store()

    batch = torch.from_numpy(batches[0][0]).float()
    N = len(batch)
    cond_model = lambda: pyro.condition(MoveState.model, {'vel_seq_{}'.format(i): batch[i] for i in range(N)})(N)

    guide = AutoGuideList(cond_model)
    guide.add(AutoDelta(poutine.block(cond_model, expose_fn=lambda msg: msg['name'].startswith('p_'))))
    guide.add(AutoDiscreteSequential(poutine.block(cond_model, expose=['state'])))
    
    svi = SVI(
        model=cond_model, 
        guide=guide,
        optim=Adam({'lr': 0.1}), 
        loss=TraceEnum_ELBO(max_plate_nesting=1))
    losses = []
    for _ in range(50):
        losses.append(svi.step())
     
    state_posterior.append(pyro.param('auto_state_probs').clone())

In [520]:
scores = np.array([p[MoveState.Slowing.value].item() for p in state_posterior])
batches_sorted = np.argsort(scores)[::-1]

In [504]:
np.sort(scores)

array([0.00327484, 0.00339094, 0.00371159, 0.00378335, 0.00471521,
       0.00487807, 0.00566505, 0.00664504, 0.00942859, 0.00976891,
       0.00989294, 0.0104405 , 0.0108022 , 0.01137481, 0.01231527,
       0.01366224, 0.01367114, 0.01470569, 0.01497779, 0.01908691,
       0.02102455, 0.0273417 , 0.03347053, 0.03867813, 0.03978542,
       0.04022329, 0.05513223, 0.0655796 , 0.08065056, 0.08223477,
       0.0890635 , 0.10028731, 0.10750953, 0.10877361, 0.1231674 ,
       0.18338451, 0.19029769, 0.22513543, 0.22552331, 0.24423394,
       0.29427391, 0.29712784, 0.30337724, 0.31935424, 0.32441095,
       0.33964455, 0.34921095, 0.36726049, 0.37692177, 0.40990138,
       0.41004571, 0.4110817 , 0.41637796, 0.41878894, 0.4217723 ,
       0.42301056, 0.42321721, 0.42372006, 0.42376339, 0.42471448,
       0.42545828, 0.43051496, 0.43144295, 0.4379659 , 0.44233748,
       0.44455779, 0.44617155, 0.4473902 , 0.4502835 , 0.45340228,
       0.45363969, 0.45550957, 0.45634079, 0.45764124, 0.45935

In [498]:
vgrid_traj([batch_traj_flat[i] for i in batches_sorted[:100]])

VGridWidget(vgrid_spec={'interval_blocks': [{'video_id': 1, 'interval_sets': [{'interval_set': [{'payload': {'…

In [450]:
vgrid_traj([batch_traj_flat[i] for i in range(len(batches_flat)) if pred_classes_flat[i] == MoveState.Slowing])

VGridWidget(vgrid_spec={'interval_blocks': [{'video_id': 1, 'interval_sets': [{'interval_set': [{'payload': {'…

In [451]:
traj_str = [
    ''.join([s.to_char() for s in l])
    for l in pred_classes
]
traj_str

['___________',
 '___________',
 '<<oooooo>o',
 '<oo',
 '>>>',
 '________',
 '________',
 '<<>>>',
 'o>o',
 '<o',
 '__________________',
 '__________________',
 '<__oooooooo>><',
 '<>>>',
 'oooooo>',
 'oo',
 '>o>',
 'oo<>>',
 'ooo>>o',
 'oo>oo',
 '>>',
 '>>oo',
 '<o>o',
 'oo>>>>',
 '<<o>>><<__>><<',
 'oo>><<_>>>>>',
 '<<<>>>>>>',
 '<o><<ooo>>>o',
 'oooo',
 '____________',
 '____________',
 '<oo<ooo>o>>',
 '__________',
 '________>o',
 '<<<oo<>>>',
 '>o',
 '<<>><<oo<o>>>o',
 'o<<o>>>oo<>>>',
 '>oo<o',
 '<o>>oo<o>>>',
 '___________',
 '___________',
 'ooooooo>oo',
 '_______________',
 '_______________',
 'oooooooo>o<<o',
 '<<>',
 'oo',
 'ooooo>>',
 'ooooo>o',
 '<>>>><',
 '______',
 '______',
 'o<o<>o',
 '______>oooo>ooo',
 '<<<ooo>>',
 '<o<<>>ooooo>',
 '<<______>oo___________>>>>>',
 '<<o>>o<<>o<<>>',
 '____o>>o>>>',
 'oooo<o>>ooooo>o>>',
 '<oooo>>',
 '__________',
 '__________',
 'oooo>oo',
 '>>o',
 'oooo>>>o',
 '<oo>o',
 '<oo>>']

In [452]:
def traj_match(pattern):
    matches = []
    for i, ts in enumerate(traj_str):
        for m in re.finditer(pattern, ts):
            matches.append((i, m.start(0), m.end(0), m.group(0)))
    return matches

def build_traj_subseq(matches):
    new_trajs = []
    for (i, j, k, _) in matches:
        traj = trajectories[i]
        traj_parts = [
            batch_traj[i][j2]
            for j2 in range(j, k)
        new_traj = copy(traj_parts[0])
        new_traj.pos = new_traj.pos[:]
        for t in traj_parts[1:]:
            new_traj.pos += t.pos[:]
        new_trajs.append(new_traj)
    return new_trajs

In [506]:
;;vgrid_traj(build_traj_subseq(traj_match('<+_+')))

VGridWidget(vgrid_spec={'interval_blocks': [{'video_id': 4, 'interval_sets': [{'interval_set': [{'payload': {'…

In [454]:
matched_traj = traj_match('<+_+')

class StoppedFor(Enum):
    Pedestrian = 1
    Vehicle = 2

def stopped_for(match):
    (traj_i, batch_start, batch_end, pattern) = match
    stop_start = pattern.index('_')
    traj = veh_trajectories[traj_i]
    video = traj.obj.video
    window_size = compute_window_size(video)
    relevant_pos = traj.pos[(batch_start + stop_start) * window_size * 3:batch_end * window_size * 3]
        
    veh_traj = [veh for veh in veh_trajectories if veh.obj.video.id == video.id]
    ped_traj = [ped for ped in ped_trajectories if ped.obj.video.id == video.id]
    
    pos = relevant_pos[0]
    frame = pos.frame    
    pos_arr = np.array([pos.x, pos.y])
    obstacle_pos = pos_arr + np.array([math.cos(relevant_pos[0].psi), math.sin(relevant_pos[0].psi)]) * 0.1
    
    def getdist(l):
        r = []
        for o in l:
            if isinstance(o, VehTraj) and o.obj.id == traj.obj.id:
                continue
            p = o.frame(frame)
            if p is not None:
                r.append([p.x, p.y])
        r = np.array(r)
        if len(r) == 0:
            return [1000000]
        return np.linalg.norm(r - pos_arr, axis=1)
    
    veh_dist = getdist(veh_traj)
    ped_dist = getdist(ped_traj)
    
    if min(veh_dist) < min(ped_dist):
        return StoppedFor.Vehicle
    else:
        return StoppedFor.Pedestrian
    
    
[stopped_for(t) for t in matched_traj]

[<StoppedFor.Pedestrian: 1>,
 <StoppedFor.Pedestrian: 1>,
 <StoppedFor.Vehicle: 2>,
 <StoppedFor.Pedestrian: 1>]